In [1]:
# from noduleAnalysis.model import *
from noduleAnalysis.prepcache import *

import datetime

from utill.util import importstr
from utill.logconf import logging


log = logging.getLogger('nb')

In [2]:
def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')  # <1>
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))  # <2>
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}).main()".format(app, argv))

PREPARE CACHE

In [4]:
run('noduleAnalysis.prepcache.LunaPrepCacheApp')

2022-06-03 00:14:44,935 INFO     pid:20060 nb:004:run Running: noduleAnalysis.prepcache.LunaPrepCacheApp(['--num-workers=4']).main()
2022-06-03 00:14:44,936 INFO     pid:20060 noduleAnalysis.prepcache:048:main Starting LunaPrepCacheApp, Namespace(batch_size=1024, num_workers=4)
2022-06-03 00:14:47,002 INFO     pid:20060 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x0000022F57280FA0>: 56928 training samples, 56816 neg, 112 pos, unbalanced ratio
2022-06-03 00:14:47,003 WARNING  pid:20060 utill.util:126:enumerateWithEstimate Stuffing cache ----/56, starting
2022-06-03 00:36:29,431 INFO     pid:20060 utill.util:147:enumerateWithEstimate Stuffing cache    8/56, done at 2022-06-03 03:19:00, 2:59:48
2022-06-03 00:59:34,101 INFO     pid:20060 utill.util:147:enumerateWithEstimate Stuffing cache   16/56, done at 2022-06-03 03:00:40, 2:41:28
2022-06-03 01:12:48,389 INFO     pid:20060 utill.util:147:enumerateWithEstimate Stuffing cache   32/56, don

CLASSIFICATION TRAINING - NODULE-NONNODULE

In [3]:
run('noduleAnalysis.training.ClassificationTrainingApp', f'--epochs=2', 'nodule-nonnodule')

2022-06-03 11:39:41,961 INFO     pid:13152 nb:004:run Running: noduleAnalysis.training.ClassificationTrainingApp(['--num-workers=4', '--epochs=2', 'nodule-nonnodule']).main()
2022-06-03 11:39:42,453 INFO     pid:13152 noduleAnalysis.training:144:initModel Using CUDA; 1 devices.
2022-06-03 11:39:42,527 INFO     pid:13152 noduleAnalysis.training:210:main Starting ClassificationTrainingApp, Namespace(batch_size=24, num_workers=4, epochs=2, dataset='LunaDataset', model='LunaModel', malignant=False, finetune='', finetune_depth=1, tb_prefix='NoduleAnalysis', comment='nodule-nonnodule')
2022-06-03 11:39:44,302 INFO     pid:13152 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x0000020F0C124BB0>: 50616 training samples, 50515 neg, 101 pos, 1:1 ratio
2022-06-03 11:39:44,322 INFO     pid:13152 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x0000020F0DF18250>: 6312 validation samples, 6301 neg, 11 pos, 

PERFORMANCE ON VALIDATION DATA

In [3]:
run('noduleAnalysis.nodule_analysis.NoduleAnalysisApp', '--run-validation')

2022-06-03 12:21:06,045 INFO     pid:21768 nb:004:run Running: noduleAnalysis.nodule_analysis.NoduleAnalysisApp(['--num-workers=4', '--run-validation']).main()
2022-06-03 12:21:19,997 DEBUG    pid:21768 noduleAnalysis.nodule_analysis:211:initModels data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state
2022-06-03 12:21:20,106 DEBUG    pid:21768 noduleAnalysis.nodule_analysis:227:initModels data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state
2022-06-03 12:21:20,123 INFO     pid:21768 noduleAnalysis.nodule_analysis:287:main Starting NoduleAnalysisApp, Namespace(batch_size=4, num_workers=4, run_validation=True, include_train=False, segmentation_path='data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state', cls_model='LunaModel', classification_path='data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', malignancy_model='LunaModel', malignancy_path=None, tb_prefix='NoduleAnalysis', series_uid=None)
2022-06-03 12:21:2

1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              612 |               90
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                0


2022-06-03 12:22:23,835 INFO     pid:21768 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001C806786070>: 1 general series, 274 slices, 1 nodules
2022-06-03 12:22:43,583 INFO     pid:21768 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001C806786460>: 2506 training samples, 2506 neg, 0 pos, unbalanced ratio


1.3.6.1.4.1.14519.5.2.1.6279.6001.130438550890816550994739120843
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1803 |              702
          Benign |                0 |                0 |                1
       Malignant |                0 |                0 |                0


2022-06-03 12:23:03,339 INFO     pid:21768 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001C806784970>: 1 general series, 125 slices, 2 nodules
2022-06-03 12:23:13,093 INFO     pid:21768 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001C8067A5940>: 1021 training samples, 1021 neg, 0 pos, unbalanced ratio


...

1.3.6.1.4.1.14519.5.2.1.6279.6001.332453873575389860371315979768
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1286 |              232
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                0


2022-06-03 12:25:14,674 INFO     pid:21768 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001C806784970>: 1 general series, 516 slices, 1 nodules
2022-06-03 12:26:08,680 INFO     pid:21768 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001C806784970>: 1869 training samples, 1869 neg, 0 pos, unbalanced ratio


1.3.6.1.4.1.14519.5.2.1.6279.6001.525937963993475482158828421281
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1363 |              504
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                1


2022-06-03 12:26:35,279 INFO     pid:21768 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001C8067867C0>: 1 general series, 127 slices, 1 nodules
2022-06-03 12:26:52,804 INFO     pid:21768 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001C806786070>: 1264 training samples, 1264 neg, 0 pos, unbalanced ratio
2022-06-03 12:27:00,940 INFO     pid:21768 utill.util:147:enumerateWithEstimate Series    8/9, done at 2022-06-03 12:27:00, 0:05:36
2022-06-03 12:27:00,941 WARNING  pid:21768 utill.util:160:enumerateWithEstimate Series ----/9, done at 2022-06-03 12:27:00
2022-06-03 12:27:00,946 INFO     pid:21768 nb:009:run Finished: noduleAnalysis.nodule_analysis.NoduleAnalysisApp.['--num-workers=4', '--run-validation']).main()


1.3.6.1.4.1.14519.5.2.1.6279.6001.752756872840730509471096155114
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1111 |              149
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                1
Total
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |            10915 |             2547
          Benign |                4 |                0 |                2
       Malignant |                0 |                1 |                4


In [5]:
run('noduleAnalysis.nodule_analysis.NoduleAnalysisApp','1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748')

2022-06-03 12:52:01,743 INFO     pid:22176 nb:004:run Running: noduleAnalysis.nodule_analysis.NoduleAnalysisApp(['--num-workers=4', '1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748']).main()
2022-06-03 12:52:01,745 DEBUG    pid:22176 noduleAnalysis.nodule_analysis:211:initModels data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state
2022-06-03 12:52:01,854 DEBUG    pid:22176 noduleAnalysis.nodule_analysis:227:initModels data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state
2022-06-03 12:52:01,886 INFO     pid:22176 noduleAnalysis.nodule_analysis:287:main Starting NoduleAnalysisApp, Namespace(batch_size=4, num_workers=4, run_validation=False, include_train=False, segmentation_path='data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state', cls_model='LunaModel', classification_path='data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', malignancy_model='LunaModel', malignancy_path=None, tb_prefix='Nod

found nodule candidates in 1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748:
nodule prob 0.644, center xyz XyzTuple(x=-156.509369, y=8.087493999999992, z=-64.75)
nodule prob 0.689, center xyz XyzTuple(x=-150.181244, y=-55.89688100000001, z=-214.75)
nodule prob 0.586, center xyz XyzTuple(x=-148.071869, y=107.22811899999999, z=-269.75)
nodule prob 0.661, center xyz XyzTuple(x=-147.368744, y=108.63436899999999, z=-242.25)
nodule prob 0.629, center xyz XyzTuple(x=-146.665619, y=109.33749399999999, z=-134.75)
nodule prob 0.610, center xyz XyzTuple(x=-146.665619, y=109.33749399999999, z=-89.75)
nodule prob 0.624, center xyz XyzTuple(x=-146.665619, y=108.63436899999999, z=-272.25)
nodule prob 0.613, center xyz XyzTuple(x=-146.665619, y=109.33749399999999, z=-212.25)
nodule prob 0.662, center xyz XyzTuple(x=-145.962494, y=109.33749399999999, z=-242.25)
nodule prob 0.751, center xyz XyzTuple(x=-145.962494, y=110.04061899999999, z=-104.75)
nodule prob 0.548, center xyz XyzTuple(x

MALIGNANCY CLASSIFIER

In [3]:
run('noduleAnalysis.training.ClassificationTrainingApp', f'--epochs=2', '--malignant', '--dataset=MalignantLunaDataset',
    '--finetune=data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state','finetune-head')

2022-06-03 14:11:23,752 INFO     pid:20264 nb:004:run Running: noduleAnalysis.training.ClassificationTrainingApp(['--num-workers=4', '--epochs=2', '--malignant', '--dataset=MalignantLunaDataset', '--finetune=data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', 'finetune-head']).main()
2022-06-03 14:11:24,894 INFO     pid:20264 noduleAnalysis.training:132:initModel finetuning from data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state, blocks head_linear
2022-06-03 14:11:24,898 INFO     pid:20264 noduleAnalysis.training:144:initModel Using CUDA; 1 devices.
2022-06-03 14:11:24,986 INFO     pid:20264 noduleAnalysis.training:210:main Starting ClassificationTrainingApp, Namespace(batch_size=24, num_workers=4, epochs=2, dataset='MalignantLunaDataset', model='LunaModel', malignant=True, finetune='data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', finetune_depth=1, tb_prefix='NoduleAnalysis', comment='finetune-he

In [6]:
run('noduleAnalysis.nodule_analysis.NoduleAnalysisApp', '--run-validation')

2022-06-03 14:21:31,922 INFO     pid:20264 nb:004:run Running: noduleAnalysis.nodule_analysis.NoduleAnalysisApp(['--num-workers=4', '--run-validation']).main()
2022-06-03 14:21:31,926 DEBUG    pid:20264 noduleAnalysis.nodule_analysis:211:initModels data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state
2022-06-03 14:21:32,114 DEBUG    pid:20264 noduleAnalysis.nodule_analysis:227:initModels data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state
2022-06-03 14:21:32,164 INFO     pid:20264 noduleAnalysis.nodule_analysis:287:main Starting NoduleAnalysisApp, Namespace(batch_size=4, num_workers=4, run_validation=True, include_train=False, segmentation_path='data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state', cls_model='LunaModel', classification_path='data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', malignancy_model='LunaModel', malignancy_path=None, tb_prefix='NoduleAnalysis', series_uid=None)
2022-06-03 14:21:3

1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              612 |               90
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                0


2022-06-03 14:21:56,790 INFO     pid:20264 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001EE9B805B20>: 1 general series, 274 slices, 1 nodules
2022-06-03 14:22:18,362 INFO     pid:20264 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001EEB6CC0CA0>: 2506 training samples, 2506 neg, 0 pos, unbalanced ratio


1.3.6.1.4.1.14519.5.2.1.6279.6001.130438550890816550994739120843
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1803 |              702
          Benign |                0 |                0 |                1
       Malignant |                0 |                0 |                0


2022-06-03 14:22:32,099 INFO     pid:20264 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001EEB4406370>: 1 general series, 125 slices, 2 nodules
2022-06-03 14:22:43,411 INFO     pid:20264 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001EEB5A08730>: 1021 training samples, 1021 neg, 0 pos, unbalanced ratio


...

1.3.6.1.4.1.14519.5.2.1.6279.6001.332453873575389860371315979768
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1286 |              232
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                0


2022-06-03 14:30:44,971 INFO     pid:20264 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001EE9B805CA0>: 1 general series, 516 slices, 1 nodules
2022-06-03 14:31:32,266 INFO     pid:20264 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001EE92A11580>: 1869 training samples, 1869 neg, 0 pos, unbalanced ratio


1.3.6.1.4.1.14519.5.2.1.6279.6001.525937963993475482158828421281
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1363 |              504
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                1


2022-06-03 14:32:20,580 INFO     pid:20264 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x000001EEB44061C0>: 1 general series, 127 slices, 1 nodules
2022-06-03 14:32:47,589 INFO     pid:20264 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x000001EEB5A08730>: 1264 training samples, 1264 neg, 0 pos, unbalanced ratio
2022-06-03 14:33:11,927 INFO     pid:20264 utill.util:147:enumerateWithEstimate Series    8/9, done at 2022-06-03 14:33:11, 0:11:39
2022-06-03 14:33:11,928 WARNING  pid:20264 utill.util:160:enumerateWithEstimate Series ----/9, done at 2022-06-03 14:33:11
2022-06-03 14:33:11,937 INFO     pid:20264 nb:009:run Finished: noduleAnalysis.nodule_analysis.NoduleAnalysisApp.['--num-workers=4', '--run-validation']).main()


1.3.6.1.4.1.14519.5.2.1.6279.6001.752756872840730509471096155114
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1111 |              149
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                1
Total
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |            10915 |             2547
          Benign |                4 |                0 |                2
       Malignant |                0 |                1 |                4


In [3]:
run('noduleAnalysis.nodule_analysis.NoduleAnalysisApp', '--run-validation', '--malignancy-path')

2022-06-03 15:20:24,499 INFO     pid:472 nb:004:run Running: noduleAnalysis.nodule_analysis.NoduleAnalysisApp(['--num-workers=4', '--run-validation', '--malignancy-path']).main()
2022-06-03 15:20:37,092 DEBUG    pid:472 noduleAnalysis.nodule_analysis:214:initModels data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state
2022-06-03 15:20:37,243 DEBUG    pid:472 noduleAnalysis.nodule_analysis:230:initModels data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state
2022-06-03 15:20:37,277 INFO     pid:472 noduleAnalysis.nodule_analysis:290:main Starting NoduleAnalysisApp, Namespace(batch_size=4, num_workers=4, run_validation=True, include_train=False, segmentation_path='data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state', cls_model='LunaModel', classification_path='data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', malignancy_model='LunaModel', malignancy_path='data\\models\\NoduleAnalysis\\mal_2022-06-03_14.11.24_f

1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260
                 |    Complete Miss |     Filtered Out |     Pred. Benign |  Pred. Malignant
     Non-Nodules |                  |              612 |               45 |               45
          Benign |                0 |                0 |                0 |                0
       Malignant |                0 |                0 |                0 |                0


2022-06-03 15:21:24,642 INFO     pid:472 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x00000217AC10F640>: 1 general series, 274 slices, 1 nodules
2022-06-03 15:21:47,005 INFO     pid:472 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x00000217AC0D3490>: 2506 training samples, 2506 neg, 0 pos, unbalanced ratio


1.3.6.1.4.1.14519.5.2.1.6279.6001.130438550890816550994739120843
                 |    Complete Miss |     Filtered Out |     Pred. Benign |  Pred. Malignant
     Non-Nodules |                  |             1803 |              367 |              335
          Benign |                0 |                0 |                0 |                1
       Malignant |                0 |                0 |                0 |                0


2022-06-03 15:22:07,318 INFO     pid:472 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x00000217AC10FCA0>: 1 general series, 125 slices, 2 nodules
2022-06-03 15:22:18,022 INFO     pid:472 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x00000217AC106220>: 1021 training samples, 1021 neg, 0 pos, unbalanced ratio


...

1.3.6.1.4.1.14519.5.2.1.6279.6001.332453873575389860371315979768
                 |    Complete Miss |     Filtered Out |     Pred. Benign |  Pred. Malignant
     Non-Nodules |                  |             1286 |              110 |              122
          Benign |                0 |                0 |                0 |                0
       Malignant |                0 |                0 |                0 |                0


2022-06-03 15:24:40,133 INFO     pid:472 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x00000217AC0E28B0>: 1 general series, 516 slices, 1 nodules
2022-06-03 15:25:15,805 INFO     pid:472 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x00000217AC1012E0>: 1869 training samples, 1869 neg, 0 pos, unbalanced ratio


1.3.6.1.4.1.14519.5.2.1.6279.6001.525937963993475482158828421281
                 |    Complete Miss |     Filtered Out |     Pred. Benign |  Pred. Malignant
     Non-Nodules |                  |             1363 |              249 |              255
          Benign |                0 |                0 |                0 |                0
       Malignant |                0 |                0 |                0 |                1


2022-06-03 15:25:34,119 INFO     pid:472 combining_data.segm_data:293:__init__ <combining_data.segm_data.Luna2dSegmentationDataset object at 0x00000217AC10F640>: 1 general series, 127 slices, 1 nodules
2022-06-03 15:25:58,098 INFO     pid:472 combining_data.analysis_data:316:__init__ <combining_data.analysis_data.LunaDataset object at 0x00000217AC1012E0>: 1264 training samples, 1264 neg, 0 pos, unbalanced ratio
2022-06-03 15:26:45,611 INFO     pid:472 utill.util:147:enumerateWithEstimate Series    8/9, done at 2022-06-03 15:26:45, 0:06:04
2022-06-03 15:26:45,612 WARNING  pid:472 utill.util:160:enumerateWithEstimate Series ----/9, done at 2022-06-03 15:26:45
2022-06-03 15:26:45,621 INFO     pid:472 nb:009:run Finished: noduleAnalysis.nodule_analysis.NoduleAnalysisApp.['--num-workers=4', '--run-validation', '--malignancy-path']).main()


1.3.6.1.4.1.14519.5.2.1.6279.6001.752756872840730509471096155114
                 |    Complete Miss |     Filtered Out |     Pred. Benign |  Pred. Malignant
     Non-Nodules |                  |             1111 |               52 |               97
          Benign |                0 |                0 |                0 |                0
       Malignant |                0 |                0 |                1 |                0
Total
                 |    Complete Miss |     Filtered Out |     Pred. Benign |  Pred. Malignant
     Non-Nodules |                  |            10915 |             1229 |             1318
          Benign |                4 |                0 |                0 |                2
       Malignant |                0 |                1 |                2 |                2


In [3]:
run('noduleAnalysis.nodule_analysis.NoduleAnalysisApp','1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748')

2022-06-03 15:29:10,150 INFO     pid:19392 nb:004:run Running: noduleAnalysis.nodule_analysis.NoduleAnalysisApp(['--num-workers=4', '1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748']).main()
2022-06-03 15:29:41,631 DEBUG    pid:19392 noduleAnalysis.nodule_analysis:214:initModels data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state
2022-06-03 15:29:41,797 DEBUG    pid:19392 noduleAnalysis.nodule_analysis:230:initModels data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state
2022-06-03 15:29:41,870 INFO     pid:19392 noduleAnalysis.nodule_analysis:290:main Starting NoduleAnalysisApp, Namespace(batch_size=4, num_workers=4, run_validation=False, include_train=False, segmentation_path='data/models/segmentation/seg_2022-05-25_11.11.48_none.best.state', cls_model='LunaModel', classification_path='data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', malignancy_model='LunaModel', malignancy_path='data/models/NoduleA

found nodule candidates in 1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748:
nodule prob 0.644, malignancy prob 0.042, center xyz XyzTuple(x=-156.509369, y=8.087493999999992, z=-64.75)
nodule prob 0.689, malignancy prob 0.482, center xyz XyzTuple(x=-150.181244, y=-55.89688100000001, z=-214.75)
nodule prob 0.586, malignancy prob 0.522, center xyz XyzTuple(x=-148.071869, y=107.22811899999999, z=-269.75)
nodule prob 0.661, malignancy prob 0.456, center xyz XyzTuple(x=-147.368744, y=108.63436899999999, z=-242.25)
nodule prob 0.629, malignancy prob 0.415, center xyz XyzTuple(x=-146.665619, y=109.33749399999999, z=-134.75)
nodule prob 0.610, malignancy prob 0.280, center xyz XyzTuple(x=-146.665619, y=109.33749399999999, z=-89.75)
nodule prob 0.624, malignancy prob 0.353, center xyz XyzTuple(x=-146.665619, y=108.63436899999999, z=-272.25)
nodule prob 0.613, malignancy prob 0.470, center xyz XyzTuple(x=-146.665619, y=109.33749399999999, z=-212.25)
nodule prob 0.662, malignancy 

FINETUNING - DEPTH 2

In [3]:
run('noduleAnalysis.training.ClassificationTrainingApp', f'--epochs=2', '--malignant', '--dataset=MalignantLunaDataset',
    '--finetune=data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state','finetune-depth=2')

2022-06-03 15:36:01,843 INFO     pid:18816 nb:004:run Running: noduleAnalysis.training.ClassificationTrainingApp(['--num-workers=4', '--epochs=2', '--malignant', '--dataset=MalignantLunaDataset', '--finetune=data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', 'finetune-depth=2']).main()
2022-06-03 15:36:02,333 INFO     pid:18816 noduleAnalysis.training:132:initModel finetuning from data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state, blocks head_linear
2022-06-03 15:36:02,368 INFO     pid:18816 noduleAnalysis.training:144:initModel Using CUDA; 1 devices.
2022-06-03 15:36:02,453 INFO     pid:18816 noduleAnalysis.training:210:main Starting ClassificationTrainingApp, Namespace(batch_size=24, num_workers=4, epochs=2, dataset='MalignantLunaDataset', model='LunaModel', malignant=True, finetune='data/models/NoduleAnalysis/cls_2022-06-03_11.39.42_nodule-nonnodule.best.state', finetune_depth=1, tb_prefix='NoduleAnalysis', comment='finetune